# Steam Sale Sensei

Predicting when a game will first go on sale. Dataset starts from a kaggle dataset to first get a list of games, and then pricing history data comes from https://docs.isthereanydeal.com

## Data Collection

In [4]:
import os
import pandas as pd
import requests
from dotenv import load_dotenv
from datetime import datetime

In [5]:
games_df = pd.read_csv('../data/games.csv')
games_df.head(3)

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,['English'],...,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"['English', 'Portuguese - Brazil']",...,0,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


Just looking at some specific games to cpoy and paste exact string to test the API.

In [6]:
for game in games_df['Name']:
    if type(game) ==str and 'Batman' in game:
        print(game)

Batman™: Arkham Origins Blackgate - Deluxe Edition
Batman: Arkham City
LEGO® Batman™ 3: Beyond Gotham
Batman: The Enemy Within - The Telltale Series
Batman™: Arkham Origins
Batman™: Arkham VR
Batman™: Arkham Knight
Batman: Arkham Asylum Game of the Year Edition
Batman: Arkham Asylum Game of the Year Edition
Batman: Arkham City - Game of the Year Edition
Batman - The Telltale Series
LEGO® Batman™: The Videogame
LEGO® Batman™ 2: DC Super Heroes


In [20]:
games_df[games_df['Name'] == 'Batman™: Arkham Knight']

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
35324,208650,Batman™: Arkham Knight,"Jun 23, 2015",2000000 - 5000000,1996,17,19.99,24,Batman™: Arkham Knight brings the award-winnin...,"['English', 'French', 'Italian', 'German', 'Sp...",...,289,870,455,Rocksteady Studios,Warner Bros. Interactive Entertainment,"Single-player,Steam Achievements,Full controll...","Action,Adventure","Action,Open World,Superhero,Stealth,Adventure,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


### Exploring IsThereAnyDeal API

The API won't give us everything we need with one simple request, so we can follow these steps:
1. Use the title or AppID of a game to look it up and get the game_id
2. Using the game_id we can get the game info, which has the release date
3. Finally we also just need the pricing history to see when it first went on sale.

Target variable will (likely) be the number of months it took before the game went on sale for the first time.

In [12]:
load_dotenv('../.env')
api_key = os.getenv('API_KEY')

In [15]:
# helper function to make requests and prevent repetitive code
def request_game_data(endpoint, params):
    response = requests.get(endpoint, params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return f'Error: {response.status_code}'

In [98]:
endpoint = 'https://api.isthereanydeal.com/games/lookup/v1'

params = {
    'key': api_key,
    'appid': 208650
    #'title': 'Batman™: Arkham Knight'
}

response = requests.get(endpoint, params=params)
if response.status_code == 200:
    # Print the response headers
    print("Response Headers:")
    for header, value in response.headers.items():
        print(f"{header}: {value}")
    
    rate_limit = response.headers.get('X-RateLimit-Limit')
    rate_remaining = response.headers.get('X-RateLimit-Remaining')
    rate_reset = response.headers.get('X-RateLimit-Reset')

    print("\nRate Limit Info:")
    if rate_limit:
        print(f"X-RateLimit-Limit: {rate_limit}")
    if rate_remaining:
        print(f"X-RateLimit-Remaining: {rate_remaining}")
    if rate_reset:
        print(f"X-RateLimit-Reset: {rate_reset}")

Response Headers:
Date: Sun, 19 May 2024 03:09:46 GMT
Content-Type: application/json
Transfer-Encoding: chunked
Connection: keep-alive
set-cookie: PHPSESSID=ha92pu3nc16pchkhn9u5ubet49; path=/
expires: Thu, 19 Nov 1981 08:52:00 GMT
Cache-Control: no-store, no-cache, must-revalidate
pragma: no-cache
x-frame-options: DENY
access-control-allow-origin: *
CF-Cache-Status: DYNAMIC
Report-To: {"endpoints":[{"url":"https:\/\/a.nel.cloudflare.com\/report\/v4?s=NxCsiJvwTX3zufxalZN6WbGtj8p5FanFf9Bmqg4pY1BX1msKig0zRm02mcx49RnprugKtotliuo53YqRJ5oAWwNJEAg90BGot4tHJq3BjsuBx705p4AXNKwjinuSawzNONw3955u6%2F8%3D"}],"group":"cf-nel","max_age":604800}
NEL: {"success_fraction":0,"report_to":"cf-nel","max_age":604800}
Server: cloudflare
CF-RAY: 8860e3d9db8f9e04-EWR
Content-Encoding: br
alt-svc: h3=":443"; ma=86400

Rate Limit Info:


Getting game id

In [80]:
endpoint = 'https://api.isthereanydeal.com/games/lookup/v1'

params = {
    'key': api_key,
    'appid': 208650
    #'title': 'Batman™: Arkham Knight'
}

game_lookup = request_game_data(endpoint, params)
print(game_lookup)


{'found': True, 'game': {'id': '018d937f-0e0c-70be-91d5-75e399e6b4e3', 'slug': 'batman-arkham-knight', 'title': 'Batman: Arkham Knight', 'type': 'game', 'mature': False}}


Getting game info for release date

In [88]:
endpoint = "https://api.isthereanydeal.com/games/info/v2"

params = {
    'key': api_key,
    'id': '018d937f-0e0c-70be-91d5-75e399e6b4e3'
}

game_info = request_game_data(endpoint, params)
print(game_info)
print(game_info['releaseDate'])

{'id': '018d937f-0e0c-70be-91d5-75e399e6b4e3', 'slug': 'batman-arkham-knight', 'title': 'Batman: Arkham Knight', 'type': 'game', 'mature': False, 'assets': {'boxart': 'https://dbxce1spal1df.cloudfront.net/018d937f-0e0c-70be-91d5-75e399e6b4e3/boxart.jpg', 'banner145': 'https://dbxce1spal1df.cloudfront.net/018d937f-0e0c-70be-91d5-75e399e6b4e3/banner145.jpg', 'banner300': 'https://dbxce1spal1df.cloudfront.net/018d937f-0e0c-70be-91d5-75e399e6b4e3/banner300.jpg', 'banner400': 'https://dbxce1spal1df.cloudfront.net/018d937f-0e0c-70be-91d5-75e399e6b4e3/banner400.jpg', 'banner600': 'https://dbxce1spal1df.cloudfront.net/018d937f-0e0c-70be-91d5-75e399e6b4e3/banner600.jpg'}, 'earlyAccess': False, 'achievements': True, 'tradingCards': True, 'appid': 208650, 'tags': ['Action', 'Open World', 'Superhero', 'Stealth', 'Third Person'], 'releaseDate': '2015-06-23', 'developers': [{'id': 423, 'name': 'Rocksteady Studios'}, {'id': 6704, 'name': 'Rocksteady'}], 'publishers': [{'id': 21, 'name': 'Warner Bros.

Getting game price history

In [91]:
endpoint_prices = 'https://api.isthereanydeal.com/games/history/v2'

since_datetime = datetime.strptime('2015-06-23 00:0:00', '%Y-%m-%d %H:%M:%S') # random release date
# since_datetime = datetime.strptime(f"{game_info['releaseDate']} 00:0:00", "%Y-%m-%d %H:%M:%S") # using release date from previous cell
since_datetime_str = since_datetime.isoformat() + '-05:00'


params = {
    'key': api_key,
    'country': 'US',
    'id': "018d937f-0e0c-70be-91d5-75e399e6b4e3",
    'shops': [61],
    'since': since_datetime_str
}

print(request_game_data(endpoint_prices, params))

[{'timestamp': '2024-04-18T19:15:49+02:00', 'shop': {'id': 61, 'name': 'Steam'}, 'deal': {'price': {'amount': 19.99, 'amountInt': 1999, 'currency': 'USD'}, 'regular': {'amount': 19.99, 'amountInt': 1999, 'currency': 'USD'}, 'cut': 0}}, {'timestamp': '2024-04-11T19:18:03+02:00', 'shop': {'id': 61, 'name': 'Steam'}, 'deal': {'price': {'amount': 3.99, 'amountInt': 399, 'currency': 'USD'}, 'regular': {'amount': 19.99, 'amountInt': 1999, 'currency': 'USD'}, 'cut': 80}}, {'timestamp': '2024-03-21T18:18:55+01:00', 'shop': {'id': 61, 'name': 'Steam'}, 'deal': {'price': {'amount': 19.99, 'amountInt': 1999, 'currency': 'USD'}, 'regular': {'amount': 19.99, 'amountInt': 1999, 'currency': 'USD'}, 'cut': 0}}, {'timestamp': '2024-03-14T18:27:41+01:00', 'shop': {'id': 61, 'name': 'Steam'}, 'deal': {'price': {'amount': 3.99, 'amountInt': 399, 'currency': 'USD'}, 'regular': {'amount': 19.99, 'amountInt': 1999, 'currency': 'USD'}, 'cut': 80}}, {'timestamp': '2024-02-26T19:21:47+01:00', 'shop': {'id': 61,

### Getting Pricing History Data

In [92]:
def get_game_id(appid, api_key, endpoint = 'https://api.isthereanydeal.com/games/lookup/v1'):
    params = {
        'key': api_key,
        'appid': appid
    }

    game_lookup = request_game_data(endpoint, params)
    
    return game_lookup['game']['id']

def get_game_info(game_id, api_key, endpoint = 'https://api.isthereanydeal.com/games/info/v2'):
    params = {
        'key': api_key,
        'id': game_id
    }

    game_info = request_game_data(endpoint, params)
    
    return game_info['releaseDate']

def get_price_history(game_id, release_date, endpoint='https://api.isthereanydeal.com/games/history/v2'):
    since_datetime = datetime.strptime(f"{release_date} 00:0:00", "%Y-%m-%d %H:%M:%S")
    since_datetime_str = since_datetime.isoformat() + '-05:00'


    params = {
        'key': api_key,
        'country': 'US',
        'id': game_id,
        'shops': [61], # 61 is Steam
        'since': since_datetime_str
    }

    price_history = request_game_data(endpoint, params)

    return price_history

In [93]:
game_id_lst = []
release_date_lst = []
price_history_lst = []

for appid in games_df['AppID']:

    game_id = get_game_id(appid, api_key)
    release_date = get_game_info(game_id, api_key)

    game_id_lst.append(game_id)
    release_date_lst.append(release_date)

    games_price_history = get_price_history(game_id, release_date)
    price_history_lst.append(games_price_history)
    break
    

In [94]:
game_id_lst

['018d937f-2cf2-72d6-9b56-242f13170c4c']

In [95]:
release_date_lst

['2008-10-21']

In [96]:
price_history_lst

[[{'timestamp': '2024-03-21T18:18:32+01:00',
   'shop': {'id': 61, 'name': 'Steam'},
   'deal': {'price': {'amount': 19.99, 'amountInt': 1999, 'currency': 'USD'},
    'regular': {'amount': 19.99, 'amountInt': 1999, 'currency': 'USD'},
    'cut': 0}},
  {'timestamp': '2024-03-14T18:16:03+01:00',
   'shop': {'id': 61, 'name': 'Steam'},
   'deal': {'price': {'amount': 9.99, 'amountInt': 999, 'currency': 'USD'},
    'regular': {'amount': 19.99, 'amountInt': 1999, 'currency': 'USD'},
    'cut': 50}},
  {'timestamp': '2024-02-11T01:48:01+01:00',
   'shop': {'id': 61, 'name': 'Steam'},
   'deal': {'price': {'amount': 19.99, 'amountInt': 1999, 'currency': 'USD'},
    'regular': {'amount': 19.99, 'amountInt': 1999, 'currency': 'USD'},
    'cut': 0}},
  {'timestamp': '2024-01-04T19:19:36+01:00',
   'shop': {'id': 61, 'name': 'Steam'},
   'deal': {'price': {'amount': 19.99, 'amountInt': 1999, 'currency': 'USD'},
    'regular': {'amount': 19.99, 'amountInt': 1999, 'currency': 'USD'},
    'cut': 0}